### Downloading required packages

## 1. Importing dataset

In [2]:
import opendatasets as od

In [3]:
link = 'https://www.kaggle.com/datasets/joyee19/studentengagement'
od.download(link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading studentengagement.zip to .\studentengagement


100%|██████████| 38.3M/38.3M [01:23<00:00, 482kB/s]


## 2. Preprocessing the data

In [5]:
import os
import shutil
import random

In [9]:
root_path = 'C:/Projects/Python/MoiMoi/studentengagement/Student-engagement-dataset/'

# Taking subfolders paths
engaged_folder_path = os.path.join(root_path, 'Engaged')
not_engaged_folder_path = os.path.join(root_path, 'Not engaged')

# Taking classes path from each subfolders
confused_file_path = os.path.join(engaged_folder_path, 'confused')
engaged_file_path = os.path.join(engaged_folder_path, 'engaged')
frustrated_file_path = os.path.join(engaged_folder_path, 'frustrated')
looking_away_file_path = os.path.join(not_engaged_folder_path, 'Looking Away')
bored_file_path = os.path.join(not_engaged_folder_path, 'bored')
drowsy_file_path = os.path.join(not_engaged_folder_path, 'drowsy')

In [10]:
# Checking the paths
print(len(os.listdir(confused_file_path)))
print(len(os.listdir(engaged_file_path)))
print(len(os.listdir(frustrated_file_path)))
print(len(os.listdir(looking_away_file_path)))
print(len(os.listdir(bored_file_path)))
print(len(os.listdir(drowsy_file_path)))


369
347
360
423
358
263


In [11]:
# Taking paths to create training and validation folders
train_dir = os.path.join(root_path, 'train')
validation_dir = os.path.join(root_path, 'validation')

# Creating folders
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Creating subfolders by collecting folders from all the existing folders
class_folders = os.listdir(engaged_folder_path) + os.listdir(not_engaged_folder_path)

# Loop to create each folder
for class_folder in class_folders:
  os.makedirs(os.path.join(train_dir, class_folder))
  os.makedirs(os.path.join(validation_dir, class_folder))

In [12]:
# Define your training and testing directories
dirs = [engaged_folder_path, not_engaged_folder_path]

# Define your train-test split ratio
split_ratio = 0.8  # 80% for training, 20% for testing

# Iterate over each class folder in your data directory
for data_dir in dirs:
  for class_folder in os.listdir(data_dir):
      class_path = os.path.join(data_dir, class_folder)

      # Collect paths of all images in the current class folder
      image_paths = [os.path.join(class_path, img) for img in os.listdir(class_path)]

      # Shuffle the image paths
      random.shuffle(image_paths)

      # Split the image paths into training and testing sets
      split_index = int(len(image_paths) * split_ratio)
      train_paths = image_paths[:split_index]
      test_paths = image_paths[split_index:]

      # Copy images to the appropriate train and test directories
      for path in train_paths:
          shutil.copy(path, os.path.join(train_dir, class_folder))
      for path in test_paths:
          shutil.copy(path, os.path.join(validation_dir, class_folder))


## 3. Model building

In [13]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.models import Sequential

In [14]:
def create_model():
  model = Sequential([
      Conv2D(32, (3,3), activation='relu', input_shape=(300,300,3)),
      MaxPooling2D(2,2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2,2),
      Conv2D(128, (3,3), activation='relu'),
      MaxPooling2D(2,2),
      Conv2D(256, (3,3), activation='relu'),
      MaxPooling2D(2,2),
      Flatten(),
      Dense(512, activation='relu'),
      Dense(256, activation='relu'),
      # Dropout(0.3)
      Dense(6, activation='softmax')
  ])

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [15]:
model = create_model()
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 128)     

## 4. Creating ImageGenerators

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

: 

In [19]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [20]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(300,300), batch_size=32, class_mode='categorical')
validation_generator = train_datagen.flow_from_directory(validation_dir, target_size=(300,300), batch_size=32, class_mode='categorical')

Found 1694 images belonging to 6 classes.
Found 426 images belonging to 6 classes.


## 5. Fitting the model

In [22]:
model.fit(train_generator, epochs = 20, validation_data= validation_generator)

Epoch 1/20
53/53 [==============================] - 128s 2s/step - loss: 1.2268 - accuracy: 0.4770 - val_loss: 0.8898 - val_accuracy: 0.6033
Epoch 2/20
53/53 [==============================] - 123s 2s/step - loss: 0.6663 - accuracy: 0.6919 - val_loss: 0.7109 - val_accuracy: 0.6620
Epoch 3/20
53/53 [==============================] - 130s 2s/step - loss: 0.6194 - accuracy: 0.7078 - val_loss: 0.4608 - val_accuracy: 0.8028
Epoch 4/20
53/53 [==============================] - 138s 3s/step - loss: 0.5254 - accuracy: 0.7645 - val_loss: 0.4882 - val_accuracy: 0.7840
Epoch 5/20
53/53 [==============================] - 150s 3s/step - loss: 0.4825 - accuracy: 0.7887 - val_loss: 0.4021 - val_accuracy: 0.8380
Epoch 6/20
53/53 [==============================] - 153s 3s/step - loss: 0.3850 - accuracy: 0.8241 - val_loss: 0.3553 - val_accuracy: 0.8474
Epoch 7/20
53/53 [==============================] - 154s 3s/step - loss: 0.3322 - accuracy: 0.8436 - val_loss: 0.4025 - val_accuracy: 0.7981
Epoch 8/20
53

In [ ]:
model.save('facial_recognizer.h5')

c:\Users\raman\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
